In [1]:
#https://stackoverflow.com/a/54594071
from IPython.display import HTML

HTML('''<script>
  function code_toggle() {
    if (code_shown){
      $('div.input').hide('500');
      $('#toggleButton').val('Display code')
    } else {
      $('div.input').show('500');
      $('#toggleButton').val('Hide Code')
    }
    code_shown = !code_shown
  }

  $( document ).ready(function(){
    code_shown=false;
    $('div.input').hide()
  });
</script>
<form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Display code"></form>''')


In [2]:
import warnings
warnings.filterwarnings('ignore')
from lat_epig.interface import makeScrapeInterface, Parseargs                    # loads the user friendly GUI
makeScrapeInterface()

# Floriano

Text(value='', description='Text 1:')

ToggleButtons(description='Operator', options=('and', 'or', 'not'), value='and')

Text(value='', description='Text 2:')

Text(value='', description='EDCS-ID:')

Text(value='', description='Publication:')

...

Button(description='Inscription Genus...', style=ButtonStyle())

SelectMultiple(description='Inscription genus / personal status:', layout=Layout(display='none'), options=(Non…

SelectMultiple(description='and not:', layout=Layout(display='none'), options=(None, 'Augusti/Augustae', 'carm…

Button(description='Get inscriptions!', style=ButtonStyle())

Output(layout=Layout(border='1px solid black'))

In [3]:
from lat_epig.map_interface import make_map_interface
make_map_interface()



Button(description='Update Data File List', style=ButtonStyle())

Dropdown(description='Data File', layout=Layout(width='50%'), options=(('2021-07-31-EDCS_72200182+term1_%-1.ts…

Text(value='', description='Map Title:')

Dropdown(description='Basemap', index=5, options=(('60 BC provinces', 'roman_empire_60_bc_provinces.shp'), ('A…

RadioButtons(description='Basemap<br/>Styling', index=1, options=(('Light Brown', False), ('Multicoloured', Tr…

...

RadioButtons(description='DPI', index=1, options=(72, 300, 600, 1200), value=300)

Checkbox(value=False, description='Inscription IDs with Points')

Checkbox(value=False, description='(PDF Only) Append Inscriptions')

Button(description='Generate New Maps!', style=ButtonStyle())

Output(layout=Layout(border='1px solid black'))

In [4]:
from lat_epig.interactive_map_interface import make_i_map_interface
make_i_map_interface()

Button(description='Update Data File List', style=ButtonStyle())

Dropdown(description='Data File', layout=Layout(width='50%'), options=(('2021-07-31-EDCS_72200182+term1_%-1.ts…

Button(description='Refresh Interactive Map!', layout=Layout(width='30%'), style=ButtonStyle())

Output(layout=Layout(border='1px solid black'))